# Analysing TEM videos with SimpliPyTEM

This project was born out of a need for rapid, easily accesible and effective analysis methods for analysis of in situ TEM videos. Python is a very powerful language for this type of analysis based on the wide accessibility of different packages to perform a range of functions on the image. However, for many cases this makes the barriers for entry much harder and the location of relevant functions difficult to find. Here I have collated a number of methods into a single python object to make it easier to deal with electron microscopy videos, particularly those collected with gatan software. 

Before working through this tutorial, I recommend looking at the micrograph tutorial, as the concepts are the same and explained in more detail there, this is mainly looking to scale image based tutorials to videos, and most of the functions involved do exactly that. 

This method is specifically designed for use of videos taken as dm4 movies, as produced from gatan direct electron detectors (eg. our k2 camera). However, once the image and pixelsize are loaded, all the functions should work just as well for other formats, To see how I have used it for a screen recording, see the accompanying tutorial: '    '*** 

Again this mainly works out of a single class, this time named 'MicroVideo'. I'll start in a similar way to before, however proceed quicker than in the previous tutorial. 

## Import dependancies

In [ ]:
from SimpliPyTEM.MicroVideo_class import *


## Initialise MicroVideo object and open MicroVideo

In [ ]:
video = MicroVideo()
print([x for x in os.listdir('.') if 'dm4' in x])
video.open_dm('gold_growth_video_holder_test_070921.dm4')

Like the Micrograph object, the MicroVideo object contains lot of useful data and useful methods, as well as the video itself. To access the video frames run: 

In [ ]:
print(video.frames)

Again, this is no use, these need to be plotted to see anything useful from them, however we can see some information about the size and length of the video:

In [ ]:
print(video.frames.shape)
# or simply:
print(video.shape)

So this is a 13-frame video, with a size of 3834 x 3702 (numpy arrays have x and y opposite from normal image programs), lets have a look at how the first frame looks: 

We can also use other video formats e.g. avi or mp4, or load from an array, here the pixelsize and pixelunit are manually inputed: 
    

In [ ]:
print([x for x in os.listdir('.') if x[-3:]=='avi'])

## Loading videos from Avi/Mp4

Videos can be easily loaded from avi's or Mp4's, however these dont have a pixelsize or pixelunit by default, and also miss the metadata. This is simply achieved as follows:

Note that the files opened here are created at the end of the notebook and are not shipped with the tutorial, so either change the filename (Output_video.___ or do the rest of the tutorial before trying this)

In [ ]:
avi_vid = MicroVideo()
avi_vid.open_video('Output_video.avi', pixelsize=0.1, pixelunit='nm')


In [ ]:
mp4_vid = MicroVideo()
mp4_vid.open_video('Output_video.mp4', pixelsize=0.1, pixelunit='nm')

In [ ]:
print(avi_vid.pixelSize)


## Showing video stills

In [ ]:
video.imshow()

Well thats not very revealing! 
Clearly each frame is quite low contrast, lets see what the histogram looks like.



In [ ]:
video.plot_histogram(sidebyside=True)

So we dont have much signal, but we an also see that the maximum pix value is 12 (the x axis maximum) despite the fact that there are very few pixels with counts above 5. 

Lets see if we can display it better using the matplotlib library using plt.imshow() whilst adding limits into it.

We can do this by adding the vmax and vmin arguments to video.imshow()

In [ ]:
video.imshow(vmax=4, vmin=0)

Well thats still quite noisy, but we can at least see the particles a bit better! This is a low dose video with a relatively fast frame rate (we'll see how fast when discussing the metadata later)!

Lets trying to see what an average (or technically a sum) of this video looks like:

In [ ]:
video.imshow(average=True)

Much better! the particles have much clearer outlines, still not plotting very well though, lets try plotting with a vmax set again. Here rather than using a built-in function, I am using numpy sum all the video frame into a single array - this works on the time/Z axis (axis 0). 

Remember though: as we are summing the frames, the vmax needs to increase as well! We can replot the histogram with the average to see what would be suitable.

In [ ]:
video.plot_histogram(imAverage=True, histAverage=True,sidebyside=True)

In [ ]:
video.imshow(average=True, vmax=30)

Well thats a bit clearer, but I'm sure we can do better! I'll come back to that later on. 


Sometimes, I have taken a video, however I want to treat it like a single image. In these instances, the to_micrograph() function can be used. 

In [ ]:
im = video.toMicrograph()

Now we have a micrograph object: 'im', we can see the type of this as follows.

In [ ]:
type(im)

Now im can be used as shown in the micrograph tutorial.

## Showing video

we can video in a jupyter notebook as follows. This method is slow but will show you the animated video, use the 'reduce_size' function to reduce the size of the video shown.

Furthermore the vmax and vmin options shown above work in the same way here. I recommend finding a good vmax/vmin using the imshow() command before making the video, as this step can take a long time. 

In [ ]:
video.show_video(reduce_size=5, vmax=5, vmin=0)

## Metadata

So like with the micrographs, metadata from dm files is automatically loaded in and can be easily accessed: 

In [ ]:
print(video.pixelSize)
print(video.pixelUnit)

In [ ]:
fps, time = video.get_exposure()

In [ ]:
date, time = video.get_date_time()

In [ ]:
print(video.fps)
print(video.AqDate)
print(video.AqTime)

If you have other video formats which include metadata and you would like this built in, feel free to request it and I will try to add it to the package

## Video transformations

Here are the useful bits.

### Averaging the video

Often in situ EM videos have very little signal in each individual frames, and so we need to average multiple frames together. There are two built in functions for this: Average_frames and Running_average.

These functions both return a new object with the frames averaged together:
   
    - Average_frames simply splits the video into groups of n frames and sums these. 
    
    - Running_average performs a 'sliding window' averaging preceedure, frames are still split into groups of n frames, however in this case these overlap with only a single frame offset. 
    


In [ ]:
Simple_average = video.Average_frames(3)


In [ ]:
Simple_average.imshow(vmax=8, vmin=0)

In [ ]:
Running_average = video.Running_average(3)
Running_average.imshow(vmax=8, vmin=0, framenumber=0)

These look the same right? That is because they are. The first frame of these both will be the same, the difference is the number of frames (and therefore the time resolution available):


In [ ]:
print('The original video has {} frames'.format(len(video)))
print('The simple average has {} frames'.format(len(Simple_average)))
print('The running average has {} frames'.format(len(Running_average)))

The advantage of running averging is that it retains some of the time resolution that is lost in averaging, however it is the changes between frames are significantly reduced by this averaging.

Here I am going to reset the variables to reduce memory space:

In [ ]:
%reset_selective -f Simple_average
%reset_selective -f Running_average


## xy bin video 

Reducing size on the XY axis can be very useful for processing times as it greatly reduces the size of the video and therefore the number of pixels, and the data included. It will also lead to increased contrast.

This is done very simply: 


In [ ]:
video_binned = video.bin()

In [ ]:
print(video_binned.frames.shape)

From here on out I will use the binned video to speed up the processing times. 

In [ ]:
video=video.bin()

### Convert to 8-bit

This scales the video between 0 and 255, more details are given in the micrograph tutorial. 

In [ ]:
# Original histogram

video.plot_histogram(sidebyside=True)

In [ ]:
video8bit = video.convert_to_8bit()

Lets check if its different! 


In [ ]:
video8bit.plot_histogram(sidebyside=True)

So we have scaled the video between 0 and 255, however we can still see the histogram is skewed to the right, with almost no pixels having a light value, now we can see this better by adding vmax/vmin into the image, however we can also use the methods below to improve this. 


## Contrast enhancement 

The MicroVideo class has the same contrast enhancement methods as the Micrograph class. 

Clip contrast is my favourite method - this simply adjusts the blackpoint and whitepoint of the image, the maximum and minimum pixel values in the image,  and scales the pixel values to these new points. 

The maximum and minimum value can be given using the maxvalue and minvalue options, however this can also be automated using the (default) saturation option. This decides what percentage of pixels are white/black, such that saturation = 0.5 means that 0.5% of pixels in the video are white and 0.5% of pixels are black. One of the minimum/maximum can also be used and the other will work with the saturation method. 



In [ ]:
video8bit_clipped = video8bit.clip_contrast(saturation=0.5)

In [ ]:
video8bit_clipped.plot_histogram(sidebyside=True)

In [ ]:
video8bit_clipped = video8bit.clip_contrast(maxvalue = 80, minvalue =0 )
video8bit_clipped.plot_histogram(sidebyside=True)

Other methods to enhance the contrast are included, these are 'enhance_contrast' using OpenCV's built in methods allowing for alpha (contrast), beta (brightness) and gamma (non-linear brightness) control. Histogram equalisation (ensuring a good spread of pixel values, or a flat histogram) is also included. 

These methods are discussed in more detail in the micrograph analysis tutorial, and work in the same way.


## Video filters

A number of video filters are available, as with the micrograph class: 

 - Median filter: performs a median filter with kernal size defined in the call (default is 3)
 
 - Gaussian filter: performs a Gaussian filter with kernal size defined in the call (default is 3)
 
 - Weiner filter: performs a Weiner filter with kernal size defined in the call (default is 5)
 
 - Low pass filter: performs a 2D fourier transform of the image and removes the  
 
 - Non-local means filter: this compares similar regions of the image and denoises by averaging across them. This is performed by openCV, and more info can be found here: https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html


The syntax for these are all the same:
    
   > filtered_video_object = video.*****_filter(strength)
    
where \**** is one of the following: 
> median 
>
> gaussian
>
> weiner
>
> low_pass
>
> NLM

The 'Strength' value is more vairable, however all except the low pass filter have default values between 3 and 11, and require odd values (because they require a n\*n kernal with a single middle value. 


For more details on each function look at the micrograph analysis tutorial or  run

   > help(video.****_filter)
   
These are all used to reduce the noise in the image in different ways, and can be effective in difference circumstances, I recommend trying out all of them, in particular the median, gaussian and low pass filters. 

In [ ]:
video_gaussian = video8bit.gaussian_filter(3)


In [ ]:
video_gaussian.plot_histogram(sidebyside=True)

In [ ]:
video_gaussian_clipped = video_gaussian.clip_contrast()
video_gaussian_clipped.plot_histogram(sidebyside=True)

I like how this looks, so I will proceed with this video and remove the remaining videos to reduce memory.

In [ ]:
video = video_gaussian_clipped

%reset_selective -f video_gaussian
%reset_selective -f video8bit
%reset_selective -f video_binned

## Add scalebar
This is super simple - just run:


In [ ]:
videoSB = video.make_scalebar()

In [ ]:
videoSB.plot_histogram()
videoSB.imshow(average=True, vmax=50,vmin=-1)

## Saving data 

We have multiple choice when saving the data, we can save a single frame, an average frame,  a sequence of images, a tif 'stack' (all frames in one file) , an .avi video and an mp4 video. These have simple syntax: 


In [ ]:
video.write_image('Video_averaged',average=True)
video.write_image('First_frame', average=False, framenumber=0)
video.write_image('Last_frame', average=False,framenumber=-1)


In [ ]:
video.save_tif_sequence(outdir='Frames')

In [ ]:
video.save_tif_stack(outdir='.')

In [ ]:
video.write_video('Output_video.mp4')
video.write_video('Output_video.avi')

# Advanced functions

While the above functions will be good enough for many examples, there are some more specific functions, I may add more functions here from time to time. 

## Video normalisation 

Often the contrast in videos change significantly over frames, which can be distracting. Contrast can be normalised across the frames using either mean or median normalisation - these ensure the mean or median values of each frames are equal, so far these have appeared to give similar results so try either and if that doesnt sort your needs, try the other



In [ ]:
normalised_video = video.Normalise_video(normtype='mean')
normalised_video = video.Normalise_video(normtype='median')

## Local normalisation 

Soon to appear! 